In [1]:
%pylab
import tensorflow as tf
import pandas as pd

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
#load data and shuffle it
train_labeled = pd.read_hdf("train_labeled.h5", "train")
train_labeled=train_labeled.iloc[np.random.permutation(len(train_labeled))]

train_unlabeled = pd.read_hdf("train_unlabeled.h5", "train")
train_unlabeled=train_unlabeled.iloc[np.random.permutation(len(train_unlabeled))]

test = pd.read_hdf("test.h5", "test")

#slice labeled set into training and validation set
validation=train_labeled.iloc[:500]
train_labeled=train_labeled.iloc[500:]

In [3]:
# head is only there to only show the first five rows. without it you'd see the whole dataset
test.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128
30000,0.304688,0.0,0.000000,0.0,0.0,0.375000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
30001,0.386719,0.0,0.320312,0.0,0.0,0.000000,0.0,0.0,0.0,0.046875,...,0.0,0.0,0.000000,0.339844,0.0,0.0,0.0,0.0,0.761719,0.0
30002,0.000000,0.0,0.984375,0.0,0.0,0.984375,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.984375,0.0,0.0,0.0,0.0,0.000000,0.0
30003,0.000000,0.0,0.941406,0.0,0.0,0.457031,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.718750,0.988281,0.0,0.0,0.0,0.0,0.000000,0.0
30004,0.984375,0.0,0.000000,0.0,0.0,0.765625,0.0,0.0,0.0,0.984375,...,0.0,0.0,0.582031,0.187500,0.0,0.0,0.0,0.0,0.000000,0.0


In [4]:
# just playing with pandas
train_labeled.describe()

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128
count,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,...,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.0,8500.000000,8500.000000
mean,4.489176,0.385443,0.000946,0.489682,0.036305,0.029081,0.490021,0.330669,0.135221,0.062739,...,0.014526,0.007788,0.503791,0.442335,0.003107,0.000713,0.079182,0.0,0.106443,0.000100
std,2.889868,0.433419,0.026455,0.431992,0.161481,0.144422,0.432953,0.420683,0.304829,0.213978,...,0.105285,0.075746,0.437911,0.431626,0.046350,0.021137,0.238484,0.0,0.274170,0.004974
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
50%,4.000000,0.082031,0.000000,0.500000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.552734,0.332031,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
75%,7.000000,0.949219,0.000000,0.984375,0.000000,0.000000,0.984375,0.839844,0.000000,0.000000,...,0.000000,0.000000,0.984375,0.980469,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
max,9.000000,0.996094,0.988281,0.996094,0.996094,0.996094,0.996094,0.996094,0.996094,0.996094,...,0.996094,0.996094,0.996094,0.996094,0.988281,0.996094,0.996094,0.0,0.996094,0.355469


In [5]:
# Format training data
DATA_x_train=train_labeled.loc[:,'x1':'x128'].values
tmp_DATA_y_=train_labeled['y'].values
DATA_y_=[]
for label in tmp_DATA_y_:
    DATA_y_.append(zeros(10))
    DATA_y_[-1][label]=1
DATA_y_

# Format validation data
valDATA_x=validation.loc[:,'x1':'x128'].values
tmp_valDATA_y_=validation['y'].values
valDATA_y_=[]
for label in tmp_valDATA_y_:
    valDATA_y_.append(zeros(10))
    valDATA_y_[-1][label]=1
    
# Format test data
DATA_x_test=test.loc[:,'x1':'x128'].values
DATA_id_test=[30000+i for i in range(len(DATA_x_test))]

In [71]:
sess.close()

In [72]:
# Start tensorflow session
sess = tf.InteractiveSession()

In [73]:
# Abstract placeholders
x = tf.placeholder(tf.float32, shape=[None, 128])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [74]:
# Initializing model parameters, W weights and b biases
#W=tf.Variable(tf.zeros([128,10]))
#b=tf.Variable(tf.zeros([10]))

W=tf.Variable(tf.truncated_normal([128,10], stddev=0.5))
b=tf.Variable(tf.truncated_normal([10], stddev=0.5))

In [83]:
# Defining my network, including the loss function, cross_entropy
sess.run(tf.initialize_all_variables())

y=tf.nn.softmax(tf.matmul(x,W)+b)
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),reduction_indices=[1]))

In [84]:
# Gradient Descent with 0.5 step, run a given amount of times. NOTE: I removed the batching part that was in the tf tutorial.
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for i in range(1000):
    #batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x:DATA_x_train,y_:DATA_y_})

# Evaluate based on the validation set that we separated in the beginning
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print(accuracy.eval(feed_dict={x:valDATA_x,y_:valDATA_y_}))

0.848


In [31]:
prediction=tf.argmax(y,1)
PRED_y=prediction.eval(feed_dict={x: DATA_x_test})

In [32]:
import csv
with open('result_2.csv', 'w') as fp:
    a = csv.writer(fp, delimiter=',')
    a.writerow(('Id','y'))
    a.writerows(zip(DATA_id_test,PRED_y))